In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.listdir('/kaggle/input/aptos2019-blindness-detection/')

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
from fastai import *
from fastai.vision import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import fastai
import warnings
warnings.filterwarnings('ignore')


In [ ]:
df_train=pd.read_csv("/kaggle/input/aptos2019-blindness-detection/train.csv")
df_train

In [ ]:
counts = df_train['diagnosis'].value_counts()
class_list = ['0', '1','2','3','4']
for i,x in enumerate(class_list):
    counts[x] = counts.pop(i)

plt.figure(figsize=(10,5))
sns.barplot(counts.index, counts.values, alpha=0.8, palette='bright')
plt.title('Distribution of Output Classes')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Target Classes', fontsize=12)
plt.show()

In [ ]:
base_image_dir = os.path.join('/kaggle/input/aptos2019-blindness-detection/')
train_dir = os.path.join(base_image_dir,'train_images/')
df = pd.read_csv(os.path.join(base_image_dir, 'train.csv'))
df['path'] = df['id_code'].map(lambda x: os.path.join(train_dir,'{}.png'.format(x)))
df = df.drop(columns=['id_code'])
df = df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
df.head(10)

In [ ]:
df_test=pd.read_csv("/kaggle/input/aptos2019-blindness-detection/test.csv")
df_test

In [ ]:
bs = 32 #smaller batch size is better for training, but may take longer
sz=224

In [ ]:
import fastai
print(fastai.__version__)

In [ ]:
tfms = get_transforms(do_flip=True,flip_vert=True,max_rotate=360,max_warp=0,max_zoom=1.1,max_lighting=0.1,p_lighting=0.5)
src = (ImageList.from_df(df=df,path="/",cols='path') #get dataset from dataset
        .split_by_rand_pct(0.2) #Splitting the dataset
        .label_from_df(cols='diagnosis') #,label_cls=FloatList) #obtain labels from the level column
      )
# data= (src.transform(tfms,size=sz,resize_method=ResizeMethod.SQUISH,padding_mode='zeros') #Data augmentation
#         .databunch(bs=bs,num_workers=4) #DataBunch
#         .normalize(imagenet_stats) #Normalize     
#        )

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install fastai==1.0.61

In [ ]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

In [ ]:
data= (src.transform(tfms,size=sz,resize_method=ResizeMethod.SQUISH,padding_mode='zeros') #Data augmentation
         .databunch(bs=bs,num_workers=4) #DataBunch
         .normalize(imagenet_stats) #Normalize     
        )

In [ ]:
data.show_batch(rows=5,figsize=(7,6))

In [ ]:
kappa = KappaScore()
kappa.weights = "quadratic"

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
from fastai.vision import *
model = EfficientNet.from_pretrained("efficientnet-b3")
learner_enet_new_b3 = Learner(data, model, metrics=[accuracy,error_rate,kappa],model_dir=".", callback_fns=ShowGraph)
#learner_enet=cnn_learner(data,model, metrics=accuracy,model_dir=".",callback_fns=ShowGraph)

In [ ]:
learner_enet_new_b3.lr_find()

In [ ]:
learner_enet_new_b3.fit_one_cycle(1, 1e-3)

In [ ]:
sz=300

In [ ]:
learner_enet_new_b3.unfreeze()

In [ ]:
data= (src.transform(tfms,size=sz,resize_method=ResizeMethod.SQUISH,padding_mode='zeros') #Data augmentation
         .databunch(bs=bs,num_workers=4) #DataBunch
         .normalize(imagenet_stats) #Normalize     
        )

In [ ]:
learner_enet_new_b3.save('stage_1_eb3_224')

In [ ]:
learner_enet_new_b3 = Learner(data, model, metrics=[accuracy,error_rate,kappa],model_dir=".", callback_fns=ShowGraph)


In [ ]:
learner_enet_new_b3.load('stage_1_eb3_224')

In [ ]:
learner_enet_new_b3.unfreeze()

In [ ]:
learner_enet_new_b3.fit_one_cycle(1,4e-4)

In [ ]:
learner_enet_new_b3.save('enet_b3_300_final')

In [ ]:
sample_df = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/sample_submission.csv')
sample_df.head()

In [ ]:
path=Path("/kaggle/input/aptos2019-blindness-detection/")

In [ ]:
learner_enet_new_b3.data.add_test(ImageList.from_df(sample_df, path, folder='test_images', suffix='.png'))

In [ ]:
import numpy as np
import pandas as pd
import os
import scipy as sp
from functools import partial
from sklearn import metrics
from collections import Counter
import json

In [ ]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4


        return X_p

    def coefficients(self):
        return self.coef_['x']



In [ ]:
val_preds, targets = learner_enet_new_b3.get_preds(DatasetType.Valid)

In [ ]:
_ , val_index = val_preds.max(1)

In [ ]:
test_preds, y = learner_enet_new_b3.get_preds(DatasetType.Test)

In [ ]:
_ , test_index = test_preds.max(1)

In [ ]:
optR = OptimizedRounder()
optR.fit(val_index, targets)
coefficients = optR.coefficients()
val_index = optR.predict(val_index, coefficients)

In [ ]:
test_index = optR.predict(test_index, coefficients)
sample_df.diagnosis = test_index.astype(int)
sample_df.to_csv('submission.csv',index=False)

In [ ]:
sample_df

In [ ]:
os.listdir('/kaggle/input/aptos2019-blindness-detection/')

In [ ]:
!find / -name submission.csv

In [ ]:
os.listdir()